In [9]:
import torch
import numpy as np
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR10      
from torch.utils.data import DataLoader
from src.temperature_scaling import ModelWithTemperature
from src.raps import split_data_set, raps_scores, raps_classification, eval_aps
from src.inception import inception_v3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dict_path = "C:\\Users\\jiayang\\ipynb\\trainedModel\\Inception_CIFAR10.pth"
model = inception_v3(pretrained=True, dict_path=dict_path).to(device)

# preprocess the images from CIFAR10
data_transform = transforms.Compose([
    transforms.ToTensor(),       
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  
])

# load images from CIFAR10
dataset = CIFAR10(root="../../data", train=False, download=True, transform=data_transform)

# Temperature Scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=5.2).to(device)
model.set_temperature(temp_scal_loader)
model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
# Hyperparameter 
lambda_ = 0.2
k_reg = 2

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_, k_reg, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, q_hat, lambda_, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Loading weights from: C:\Users\jiayang\ipynb\trainedModel\Inception_CIFAR10.pth
Files already downloaded and verified
Before temperature - NLL: 0.416, ECE: 0.053
Optimal temperature: 5.128
After temperature - NLL: 0.766, ECE: 0.340
RAPS Classification, Start!

Running experiment 1/10...
q_hat = 0.6562867224216462
Total set size: 7498
Total coverage sets: 4476
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.4996
Average Coverage Rate in runs 1: 0.8952

Running experiment 2/10...
q_hat = 0.6541639685630799
Total set size: 7539
Total coverage sets: 4463
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.5078
Average Coverage Rate in runs 2: 0.8926

Running experiment 3/10...
q_hat = 0.652672201395035
Total set size: 7521
Total coverage sets: 4467
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.5042
Average Coverage Rate in runs 3: 0.8934

Running experiment 4/10...
q_hat = 0.6568251013755799
Total set s

## Result

- Final Average **Prediction Set Size: 1.51 / 10**
- Final Average **Coverage: 89.89% ($\alpha$=0.1)**